In [23]:
from requests import get, Session, request
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import sqlite3
from time import time, localtime, sleep, strftime, mktime
from datetime import datetime, timedelta
import json

session = Session()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
         'Referer': 'https://www.yna.co.kr/'}

In [24]:
def robotParser(domain):
    url = urlunparse(urlparse(domain)[:2] + ('',)*4)
    url += '/robots.txt'    
    pathEnable = dict()
    resp = get(url)
    if resp.status_code == 200:
        agent = None
        for line in resp.text.splitlines():
            k, *v = line.split(':')
            k = k.strip()
            v = ':'.join(v).strip()
            if k.lower() == 'user-agent':
                agent = v
                if v not in pathEnable:
                    pathEnable[v] = dict()
            else:
                if k.lower() == 'allow':
                    pathEnable[agent][v] = True
                else:
                    pathEnable[agent][v] = False
    else:
        pathEnable['*'] = True
    return pathEnable

In [25]:
def canFetch(pathEnable, path):
    agent = '*'
    path = urlparse(path).path
    
    if agent in pathEnable:
        if path in pathEnable[agent]:
            return pathEnable[agent][path]
        else:
            if path == '/':
                return True
            else:
                return canFetch(pathEnable,
                                '/'.join(path.split('/')[:-1]))
    else:
        return True

In [26]:
from google.colab import drive
drive.mount('/content/drive')
!cd '/content/drive/MyDrive/Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# Initial parameters

searchDate = '20221201'
nday = 365
page_no = 1

# url = 'https://www.yna.co.kr/search/index'
today = datetime.today()
today = datetime.strftime(today, '%Y%m%d')

seens = dict()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

query = dict()

categories = {'정치': '01', '경제': '02', '사회': '05', '금융증권산업': '03%2C04', '사건사고': '06', '문화': '07', '생활건강': '08', 'IT과학' : '09', '북한': '10', '국제': '11', '스포츠': '12', '연예': '0712'}

params = {
    'query': '%ED%96%88%EB%8B%A4',
    'period': '1y',
    'page_no': page_no,
    'ctype': 'A',
    'from': today,
    'to': today,
    'page_size': 10,
    'channel': 'basic_kr'
}

In [31]:
# crawl to db
con = sqlite3.connect('news_Crawl')
cur = con.cursor()

In [ ]:
# # create table
# # use once
# cur.executescript('''
# 	DROP TABLE IF EXISTS visited; 
# 	CREATE TABLE visited
# 	(id TEXT PRIMARY KEY,
# 	visited INTEGER NOT NULL DEFAULT 0);
  
#   DROP TABLE IF EXISTS news; 
#   CREATE TABLE news
# 	(url TEXT PRIMARY KEY,
#   id TEXT NOT NULL,
#   title TEXT,
#   content TEXT,
#   dateNews DATETIME,
#   keywords TEXT,
#   category TEXT,
#   dateInput DATETIME DEFAULT CURRENT_TIMESTAMP),
#   FOREIGN KEY(id) REFERENCES visited (id)	);
# ''')

In [ ]:
# contents_id = 'AKR20221205120300009'
# page_url = 'https://www.yna.co.kr/view/{}'.format(contents_id)
# page_resp = get(page_url, headers=headers)
# dom = BeautifulSoup(page_resp.text, 'html.parser')
# category = dom.select_one('meta[property="article:section"]')['content']
# title = dom.select_one('h1.tit')

# content = dom.select('article.story-news.article p')[2:-2]
# #### 질문? 불필요한 tag 찾아서 정리하기

# [ar.string for ar in content if ar.string is not None]
# url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=1y&from=20211206&to=20221206&ctype=A&page_size=10&channel=basic_kr'.format(page_no)
# resp = get(url, headers=headers)
# ids = json.loads(resp.text)['KR_ARTICLE']['result']
# type(ids[0]['CONTENTS_ID'])

str

In [ ]:
while True:
    url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=1y&from=20211201&to=20221201&ctype=A&page_size=10&channel=basic_kr'.format(page_no)
    resp = get(url, headers=headers)
    if resp.status_code != 200:
        break

    ids = json.loads(resp.text)['KR_ARTICLE']['result']

    for articleId in ids:
        idd = articleId['CONTENTS_ID']
        
        cur.execute('''
        INSERT OR IGNORE INTO visited (id) VALUES (:idd);
        ''', [idd])
        result = cur.fetchone()
    page_no += 1
    if page_no % 10 == 0:
        print(page_no)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030


In [ ]:
# save article ids
con.commit()

In [ ]:
for articleId in ids:
    contents_id = articleId['CONTENTS_ID']
    if contents_id in seens:
        break
    else:
        seens[contents_id] = articleId['DIST_DATE']
# get each pages
    page_url = 'https://www.yna.co.kr/view/{}'.format(contents_id)
    page_resp = get(page_url, headers=headers)
    dom = BeautifulSoup(page_resp.text, 'html.parser')
    category = dom.select_one('meta[property="article:section"]')['content']
    title = dom.select_one('h1.tit').string
    # dateNews = dom.select_one('#newsUpdateTime01')['data-pulished-time']
    # if dateNews != articleId['DIST_DATE']+articleId['DIST_TIME']:
    #     print('different date at: ', contents_id)

    # get rid of not content texts(info of journalist)
    content = dom.select('article.story-news.article p')[2:-2]
    content = [ar.string for ar in content if ar.string is not None]
    content = ''.join(content)
    query['URL'] = page_url
    query['title'] = title
    query['content'] = content
    query['dateNews'] = articleId['DIST_DATE']
    query['category'] = category
    print(query)
    cur.execute('''
        INSERT INTO news
        (URL, title, content, dateNews, category)
        VALUES (:URL, :title, :content, :dateNews, :category);
        ''', query)

    result = cur.fetchone()

{'URL': 'https://www.yna.co.kr/view/AKR20221201085351001', 'title': '野 "李해임안 반드시 가결"…본회의 개의 불발에 김의장 비판도(종합)', 'content': ' (서울=연합뉴스) 고상민 박경준 정윤주 기자 = 더불어민주당은 1일 이상민 행정안전부 장관 해임건의안 처리를 다짐하며 대여(對與) 압박을 이어갔다. 다만 김진표 국회의장이 이날 예정됐던 본회의를 개의하지 않은 탓에 2일에 해임건의안을 표결하겠다던 애초 계획에 차질이 불가피해졌다. 박홍근 원내대표는 이날 오전 정책조정회의에서 "민주당은 본회의를 반드시 열어 해임건의안을 가결할 것"이라며 "이 장관이 자진해서 사퇴하지 않거나 윤석열 대통령이 (박진 외교부 장관 해임건의에 이어) 또다시 거부한다면 탄핵소추안을 발의해 정기국회 내에 이 장관 문책을 매듭짓겠다"고 말했다.  이어 김 의장을 향해 "본회의 개의는 국회의장의 결단에 따라 가능하다"면서 "안건 없이도 본회의를 개의한 적이 있다는 사실을 의장도 잘 알 것으로 믿는다"고 했다. 법안심사 지연 등으로 본회의에 올릴 다른 안건이 부재하다는 점을 들어 김 의장이 \'해임건의안 보고\' 자체를 거부할 수도 있는 만큼 김 의장을 상대로 강한 압박에 나섰던 것이다. 박 원내대표는 오후에도 김 의장을 찾아가 같은 의견을 전달했으나 김 의장은 끝내 본회의를 열지 않았다. 민주당은 격앙된 반응을 보였다. 박 원내대표는 오후 늦게 기자간담회를 열어 "김 의장 결정에 심각한 우려를 표한다"며 "여야가 합의한 일정을 일방적으로 파기한 것은 월권이자 권한남용"이라고 주장했다. 이어 "이태원 참사 책임을 볼모로 잡아 합의된 일정을 파기한 여당에 의장까지 부화뇌동하나"라며 "예산안 처리 법정시한인 2일 오후에는 본회의를 반드시 열고, 내주 월요일까지 해임건의안 처리를 위한 본회의를 추가로 소집하라"고 촉구했다.(서울=연합뉴스) 더불어민주당 위성곤 의원(오른쪽)과 이수진 의원이 30일 국회 의안과에 이상민 행정안전부 장관 

IntegrityError: NOT NULL constraint failed: news.id

In [ ]:
cur.fetchall()
con.commit()
con.close()